# Borrower Reliability Study

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model - a special system that evaluates the ability of a potential borrower to return a loan to a bank.

## Step 1. Let's open the data file and study the general information

In [2]:
import pandas as pd

df = pd.read_csv("/datasets/data.csv")
df.head(10)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Step 2. Data preprocessing

### Pass processing

In [3]:
# getting general information about data in df table
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# skip count
print(df.isna().sum())


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Gaps in "days_employed" and "total_income" columns
I think that this is no accident, because not everyone wants to say how much they earn and how much they have worked at work.
Because the number of passes total_income and days_employed are the same
Let's replace these values ​​with averages, although ideally you need to know these data, because they may affect the study.

In [5]:
ti_avg = df["total_income"].mean()
df["total_income"] = df["total_income"].fillna(value=ti_avg)
de_avg = df["days_employed"].mean()
df["days_employed"] = df["days_employed"].fillna(value=de_avg)

In [6]:
# count the number of passes again
print(df.isna().sum())


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Replace gaps with averages**

### Replacing the data type

In [7]:
df["days_employed"] = df["days_employed"].astype("int")
df["total_income"] = df["total_income"].astype("int")



In [8]:
# check
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**We have replaced the real data type with the integer type "astype"**

### Duplicate Handling

In [9]:
# count the number of obvious duplicates
print(df.duplicated().sum())


54


In [10]:
# delete them
df = df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum())

0


**Duplicates can appear for various reasons. For example, a technical error or a human factor.**

### Lemmatization

In [11]:
# Importing a library for lemmatization of the loan purposes column and for counting
from pymystem3 import Mystem
from collections import Counter
# create an empty list where we put our lemmatization values
list = []
m = Mystem()
# create a loop and go through each value from the column, and then count which words were more
for el in df["purpose"]:
    lemma = m.lemmatize(el)
    list.extend(lemma)
count_lemmas = Counter(list)
sorted(count_lemmas.items(), key = lambda pair:pair[1], reverse=True)


[(' ', 33596),
 ('\n', 21471),
 ('недвижимость', 6353),
 ('покупка', 5900),
 ('жилье', 4461),
 ('автомобиль', 4308),
 ('образование', 4014),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2335),
 ('свой', 2231),
 ('на', 2228),
 ('строительство', 1879),
 ('высокий', 1374),
 ('получение', 1315),
 ('коммерческий', 1312),
 ('для', 1290),
 ('жилой', 1231),
 ('сделка', 941),
 ('дополнительный', 907),
 ('заниматься', 904),
 ('проведение', 773),
 ('сыграть', 769),
 ('сдача', 652),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

In order to categorize data by goals, you first need to derive the most important goals, and simplify them

In [12]:
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian') 
#we do not take the word purchase, since this word can be used for everything.
final_list = ["недвижимость", "жилье", "автомобиль", "образование", "свадьба"]

stemmed_purposes = [russian_stemmer.stem(w) for w in final_list]
print(stemmed_purposes)

['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']


In [13]:
# now numbering the dictionary
dict_stemmed_purposes = {v:k for k,v in enumerate(stemmed_purposes)}
dict_stemmed_purposes["недвижим"] = 1
print(dict_stemmed_purposes)

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}


In [14]:
# create table purspose_cat
df["purpose_cat"]=0
for row in range(len(df)):
    for purp in dict_stemmed_purposes:
        if purp in df.iloc[row,11]:
            df.iloc[row,12]= dict_stemmed_purposes[purp]
    
df.head()    
    
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4


# Here we have already categorized the data by purpose

**We can say the top 5 things people take out a loan for (we will remove the word purchase from here, since it can be used for everything):
      1. Real estate
      2 Housing
      3. Car
      4. Education
      5. Wedding**

In [15]:
# Take the value of days_employed and total_income of the abs() methods
df[["days_employed", "total_income"]] = df[["days_employed", "total_income"]].abs()
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


### Data categorization

  Categorize data separately by column

In [16]:
#Create a function salary_cat() and categorize data by income
def salary_cat(row):
    if row["total_income"]<=50000:
        return("poor")
    if 50000<row["total_income"]<=100000:
        return("normal")
    if 10000<row["total_income"]<=200000:
        return("rich")
    if row["total_income"]>200000:
        return("wealthy")

#Create a children_cat() function and categorize data by having children
def children_cat(row):
    if row["children"] == 0:
        return("нет детей")
    elif 1 <= row['children'] <= 2:
        return("1-2 ребенка")
    else:
        return("многодетная семья")
    

    

    

# create a column and apply the function
df["salary_cat"] = df.apply(salary_cat, axis=1)
df["children_cat"] = df.apply(children_cat, axis=1)


df.head(10)
 
  


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,salary_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,wealthy,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,rich,1-2 ребенка
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,rich,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,wealthy,многодетная семья
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,rich,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1,wealthy,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,wealthy,нет детей
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,3,rich,нет детей
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4,normal,1-2 ребенка
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,rich,нет детей


In [17]:
df_example = df.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
-1,47,1,2.13%,97.87%
0,14107,1063,7.54%,92.46%
1,4809,444,9.23%,90.77%
2,2052,194,9.45%,90.55%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%
20,76,8,10.53%,89.47%


**We created a column with our decisions on issuing a loan**

## Step 3 3. Data analysis

- Is there a relationship between having children and repaying a loan on time?

In [16]:
# group the table by having children and debt
children = df.groupby("children_cat")["debt"].mean().to_frame().sort_values(by="debt")
display(children)

,debt
children_cat,
нет детей,0.075353
многодетная семья,0.079523
1-2 ребенка,0.092989


**Most of all debtors are among people who have 1-2 children.**

- Is there a relationship between marital status and loan repayment on time?

In [17]:
# see what marital status options are available
df["family_status"].unique()


array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [19]:
# do the same as in the first hypothesis
#family_status = df.groupby("family_status")["debt"].mean().to_frame().sort_values(by="debt")
#display(family_status)

df_example1 = df.pivot_table(index="family_status", values="debt", aggfunc=["count", "sum", "mean", lambda x: 1-x.mean()])
df_example1.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example1.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
family_status,,,,
Не женат / не замужем,2810,274,9.75%,90.25%
в разводе,1195,85,7.11%,92.89%
вдовец / вдова,959,63,6.57%,93.43%
гражданский брак,4163,388,9.32%,90.68%
женат / замужем,12344,931,7.54%,92.46%


**Here we see that people who are not married have the most debts.**

- Is there a relationship between income and loan repayment on time?

In [19]:
# group it with debtors using the groupby() method
salary = df.groupby("salary_cat")["debt"].mean().to_frame().sort_values(by="debt")
display(salary)

,debt
salary_cat,
poor,0.061828
wealthy,0.070667
normal,0.080909
rich,0.086166


**Oddly enough, the least debtors are those who have an income of less than 50,000. And most of all, those who have an income of 100,000 to 200,000.
Apparently, because they are given loans more often.**

- How do different purposes of a loan affect its repayment on time?


In [20]:
# Group debtors by goals and loans and display
purpose = df.groupby("purpose_cat")["debt"].mean().to_frame().sort_values(by="debt")

display(purpose)


,debt
purpose_cat,
1,0.072314
4,0.079657
3,0.092177
2,0.093547


**We see that most of the debtors are those who take out a loan for education. And the most responsible are those who take out a loan for real estate transactions**

## Step 4. Conclusion

At the beginning of the project, we filled in the gaps with the average values ​​that were in income and seniority.
 It seems to me that all the same, the omissions there were not due to errors, but because not everyone wants to say their income. Therefore, for the accuracy of research, it is better to seek help in such cases.
 Next, we lemmatized the words, then made a volume of 5 popular loan goals out of them, sematized and numbered them. Then we added this data to the table, thereby categorizing the data by purpose.

We tested four hypotheses and found:
1. Is there a relationship between having children and repaying a loan on time?
    We see that the most debtors are people who have 1-2 children.
2. Is there a relationship between marital status and loan repayment on time?
    We see that unmarried couples have the most debts.
3. Is there a relationship between income level and loan repayment on time?
    We see that people with an income of less than 500,000 have the least debt. And the middle class has the most.
4. How do different purposes of a loan affect its repayment on time?
    People who take out loans for education have the most debts, and those who take loans for real estate have the least.